In [1]:
import rospy
import moveit_commander
import geometry_msgs.msg
from copy import deepcopy
from control_msgs.msg import GripperCommand
from sensor_msgs.msg import JointState
from std_msgs.msg import String as StringMsg
rospy.init_node('notebook')

In [ ]:
gripper_cmd = rospy.Publisher('/robot/gripper/command', GripperCommand, queue_size=1)
def open_gripper():
    cmd = GripperCommand()
    cmd.position = 1.0
    cmd.max_effort = 1.0
    gripper_cmd.publish(cmd)
    rospy.sleep(0.5)
def close_gripper():
    cmd = GripperCommand()
    cmd.position = 0.0
    cmd.max_effort = 1.0
    gripper_cmd.publish(cmd)
    rospy.sleep(0.5)
say_pub = rospy.Publisher('/say', StringMsg, queue_size=1)
def say(text):
    msg = StringMsg()
    msg.data = text
    say_pub.publish(msg)

def wait_for_external_effort(min_effort=3.0):
    recent_efforts = []
    import threading
    condition = threading.Condition()
    def callback(data):
        # detect and signal when effort in some joint differs above threshold from median over last second
        recent_efforts.append((data.header.stamp, data.effort))
        # keep only last second of efforts
        cutoff_time = rospy.Time.now() - rospy.Duration(1.0)
        while recent_efforts and recent_efforts[0][0] < cutoff_time:
            recent_efforts.pop(0)
        if len(recent_efforts) < 25:
            return
        # compute median effort for each joint
        import numpy as np
        efforts_array = np.array([e for t, e in recent_efforts])
        median_efforts = np.median(efforts_array, axis=0)
        # check if any joint effort differs from median by more than min_effort
        for effort in data.effort:
            if abs(effort - median_efforts[data.effort.index(effort)]) > min_effort:
                # signal condition
                with condition:
                    say("beep")
                    condition.notify_all()
                return
    
    sub = rospy.Subscriber('/robot/joint_states', JointState, callback)
    # wait for signal, timeout after 8 seconds
    with condition:
        condition.wait(timeout=8.0)
    sub.unregister()

In [3]:
mgc = moveit_commander.MoveGroupCommander("arm")
mgc.set_max_velocity_scaling_factor(1.0)
mgc.set_max_acceleration_scaling_factor(1.0)

1765380405.088479006: /moveit_python_wrappers_1765380404926213433.ros.moveit_core.robot_model: Loading robot model 'sawyer'...
1765380405.114506897: /moveit_python_wrappers_1765380404926213433.ros.moveit_core.robot_model: No root/virtual joint specified in SRDF. Assuming fixed joint
1765380408.218115894: /moveit_python_wrappers_1765380404926213433.ros.moveit_ros_planning_interface.move_group_interface: Ready to take commands for planning group arm.


1765380405.114687968: /moveit_python_wrappers_1765380404926213433.ros.moveit_core.robot_model.empty_collision_geometry: Link torso has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.


In [4]:
psi= moveit_commander.PlanningSceneInterface()

In [5]:
table_pose = geometry_msgs.msg.PoseStamped()
table_pose.header.frame_id = "base"
table_pose.pose.position.x = 0.75
table_pose.pose.position.y = 0.0
table_pose.pose.position.z = -0.28
table_pose.pose.orientation.w = 1.0
psi.add_box("table", pose=table_pose, size=(0.8, 1.1, 0.2))

In [6]:
handover_posture = JointState()
handover_posture.name = ["right_j0", "right_j1", "right_j2", "right_j3", "right_j4", "right_j5", "right_j6"]
handover_posture.position = [-0.24144140625, 1.0161748046875, -0.1290107421875, -2.0135302734375, -0.1491611328125, 1.2109267578125, -0.000138671875]

In [7]:
cubes= dict()

In [8]:
color="red"
cube = geometry_msgs.msg.PoseStamped()
cube.header.frame_id = "base"
cube.pose.position.x = 0.697
cube.pose.position.y = -0.369
cube.pose.position.z = -0.152
cube.pose.orientation.x = -0.188
cube.pose.orientation.y = 0.981
cube.pose.orientation.z = -0.020
cube.pose.orientation.w = 0.035
cubes[color] = cube

In [9]:
color = "blue"
cube = geometry_msgs.msg.PoseStamped()
cube.header.frame_id = "base"
cube.pose.position.x = 0.664
cube.pose.position.y = -0.057573
cube.pose.position.z = -0.15173
cube.pose.orientation.x = 0.186621
cube.pose.orientation.y = -0.97907
cube.pose.orientation.z = 0.0805411
cube.pose.orientation.w = 0.0103705
cubes[color] = cube

In [10]:
color = "orange"
cube = geometry_msgs.msg.PoseStamped()
cube.header.frame_id = "base"
cube.pose.position.x = 0.634108
cube.pose.position.y = 0.186313
cube.pose.position.z = -0.154222
cube.pose.orientation.x = 0.510415
cube.pose.orientation.y = -0.851931
cube.pose.orientation.z = 0.113435
cube.pose.orientation.w = 0.028699
cubes[color] = cube

In [11]:
color = "wooden"
cube = geometry_msgs.msg.PoseStamped()
cube.header.frame_id = "base"
cube.pose.position.x = 0.91436
cube.pose.position.y = -0.27065
cube.pose.position.z = -0.14875
cube.pose.orientation.x = -0.23652
cube.pose.orientation.y = 0.94525
cube.pose.orientation.z = 0.010588
cube.pose.orientation.w = 0.2246
cubes[color] = cube

In [12]:
color = "green"
cube = geometry_msgs.msg.PoseStamped()
cube.header.frame_id = "base"
cube.pose.position.x = 0.876844
cube.pose.position.y = -0.0194747
cube.pose.position.z = -0.155714
cube.pose.orientation.x = -0.0274387
cube.pose.orientation.y = 0.996718
cube.pose.orientation.z = -0.0109193
cube.pose.orientation.w = 0.0753798
cubes[color] = cube

In [13]:
color = "yellow"
cube = geometry_msgs.msg.PoseStamped()
cube.header.frame_id = "base"
cube.pose.position.x = 0.787161
cube.pose.position.y = 0.28444
cube.pose.position.z = -0.149635
cube.pose.orientation.x = -0.292078
cube.pose.orientation.y = 0.955773
cube.pose.orientation.z = -0.0124869
cube.pose.orientation.w = 0.0321393
cubes[color] = cube

In [14]:
def move_to_handover():
    mgc.set_joint_value_target(handover_posture)
    mgc.go(wait=True)
def move_to_pose(pose):
    mgc.set_pose_target(pose)
    mgc.go(wait=True)
def ready():
    mgc.set_named_target("ready")
    mgc.go(wait=True)

In [15]:
move_to_handover()

In [16]:
def place_cube(color):
    if color not in cubes:
        raise ValueError(f"No pose defined for color '{color}'")
    move_to_handover()
    say(f"Please hand me the {color} cube.")
    wait_for_external_effort()
    close_gripper()
    t = deepcopy(cubes[color])
    t.pose.position.z += 0.05
    move_to_pose(t)
    move_to_pose(cubes[color])
    open_gripper()
    move_to_pose(t)

In [17]:
open_gripper()
ready()
say("Alright, we have to arrange all these blocks properly. Please pass them to me one by one.")
rospy.sleep(rospy.Duration(4.0))
import random
todo = list(cubes.keys())
random.shuffle(todo)
for color in todo:
    place_cube(color)
ready()
say("All done, thank you for helping me out!")